In [1]:
# Importing the Libraries
#The code initializes the environment for a deep learning task in Python using PyTorch.
#It imports necessary libraries and sets the computation to run on GPU if available, or CPU otherwise, for efficient processing.
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [2]:
# Define a basic transform
#This code defines a transformation pipeline for image data,
#converting images to PyTorch tensors and normalizing their pixel values with mean 0.5 and standard deviation 0.5 for each RGB channel.
transform = transforms.Compose([
	transforms.ToTensor(),
	transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [3]:
# Load the Dataset
#This code creates a training dataset and a data loader for the CIFAR-10 dataset in PyTorch.
# The datasets.CIFAR10 function loads the CIFAR-10 dataset, specifying a directory for the data, enabling training data (train=True),
# downloading the data if not present (download=True), and applying the previously defined transformations (transform=transform).
# The DataLoader wraps the dataset, setting a batch size of 32 (number of images processed in one step) and
# shuffling the data (shuffle=True) for each training iteration to promote model generalization.
train_dataset = datasets.CIFAR10(root='./data',\
			train=True, download=True, transform=transform)
dataloader = torch.utils.data.DataLoader(train_dataset, \
								batch_size=32, shuffle=True)


100%|██████████| 170498071/170498071 [00:05<00:00, 29076196.67it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


In [4]:
# Hyperparameters

# This code defines hyperparameters for training a machine learning model, likely a GAN.
# latent_dim is the dimensionality of the input noise vector (100),
#  lr is the learning rate (0.0002) for the optimizer, beta1 and beta2 are parameters
#  for the Adam optimizer influencing momentum (0.5) and scaling (0.999), and num_epochs sets the number of training cycles through the entire dataset (10).
latent_dim = 100
lr = 0.0002
beta1 = 0.5
beta2 = 0.999
num_epochs = 10


In [5]:

#This code defines a Generator class for a GAN using PyTorch.
#It extends nn.Module and initializes a sequential model transforming a latent space vector into an image.
#The process involves linear expansion, reshaping, upscaling, and convolutional layers with ReLU activation and batch normalization,
#finally outputting an image with a Tanh activation function. The forward method defines the data flow through the network.

# Define the generator
class Generator(nn.Module):
	def __init__(self, latent_dim):
		super(Generator, self).__init__()

		self.model = nn.Sequential(
			nn.Linear(latent_dim, 128 * 8 * 8),
			nn.ReLU(),
			nn.Unflatten(1, (128, 8, 8)),
			nn.Upsample(scale_factor=2),
			nn.Conv2d(128, 128, kernel_size=3, padding=1),
			nn.BatchNorm2d(128, momentum=0.78),
			nn.ReLU(),
			nn.Upsample(scale_factor=2),
			nn.Conv2d(128, 64, kernel_size=3, padding=1),
			nn.BatchNorm2d(64, momentum=0.78),
			nn.ReLU(),
			nn.Conv2d(64, 3, kernel_size=3, padding=1),
			nn.Tanh()
		)

	def forward(self, z):
		img = self.model(z)
		return img


In [6]:
# Class for Discriminator
# Define the discriminator
#This code defines a Discriminator class, part of a Generative Adversarial Network (GAN) in PyTorch.
#It extends nn.Module and comprises a neural network that classifies images as real or fake.
#The network uses convolutional layers with LeakyReLU activation and dropout for regularization, batch normalization for stability,
#and a final linear layer with a sigmoid activation to output a probability indicating the image's authenticity.
#The forward method specifies the data flow through this network.

class Discriminator(nn.Module):
	def __init__(self):
		super(Discriminator, self).__init__()

		self.model = nn.Sequential(
		nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1),
		nn.LeakyReLU(0.2),
		nn.Dropout(0.25),
		nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1),
		nn.ZeroPad2d((0, 1, 0, 1)),
		nn.BatchNorm2d(64, momentum=0.82),
		nn.LeakyReLU(0.25),
		nn.Dropout(0.25),
		nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
		nn.BatchNorm2d(128, momentum=0.82),
		nn.LeakyReLU(0.2),
		nn.Dropout(0.25),
		nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
		nn.BatchNorm2d(256, momentum=0.8),
		nn.LeakyReLU(0.25),
		nn.Dropout(0.25),
		nn.Flatten(),
		nn.Linear(256 * 5 * 5, 1),
		nn.Sigmoid()
	)

	def forward(self, img):
		validity = self.model(img)
		return validity


In [7]:
# Building the GAN

# Define the generator and discriminator
#This code initializes a GAN's Generator and Discriminator on a specified device (GPU/CPU).
#It sets up a binary cross-entropy loss function for training.
#Adam optimizers are created for both models with predefined learning rates and momentum parameters,
#optimizing the network's performance during training.
generator = Generator(latent_dim).to(device)
discriminator = Discriminator().to(device)

# Loss function
adversarial_loss = nn.BCELoss()

# Optimizers
optimizer_G = optim.Adam(generator.parameters()\
						, lr=lr, betas=(beta1, beta2))
optimizer_D = optim.Adam(discriminator.parameters()\
						, lr=lr, betas=(beta1, beta2))


In [ ]:
# Training loop

# This code is a training loop for a Generative Adversarial Network (GAN) in PyTorch.
#  It alternates between training the Discriminator to distinguish real from fake images and training the Generator to create increasingly realistic images.
#  Each epoch processes batches of real images, computes losses for both models, and updates them using backpropagation.
#  The progress is monitored by printing losses, and at the end of each epoch, a set of generated images is displayed to visually evaluate
#  the Generator's performance.
# This iterative process aims to enhance the Generator's image creation and the Discriminator's classification accuracy.
for epoch in range(num_epochs):
	for i, batch in enumerate(dataloader):
	# Convert list to tensor
		real_images = batch[0].to(device)

		# Adversarial ground truths
		valid = torch.ones(real_images.size(0), 1, device=device)
		fake = torch.zeros(real_images.size(0), 1, device=device)

		# Configure input
		real_images = real_images.to(device)

		# ---------------------
		# Train Discriminator
		# ---------------------

		optimizer_D.zero_grad()

		# Sample noise as generator input
		z = torch.randn(real_images.size(0), latent_dim, device=device)

		# Generate a batch of images
		fake_images = generator(z)

		# Measure discriminator's ability
		# to classify real and fake images
		real_loss = adversarial_loss(discriminator\
									(real_images), valid)
		fake_loss = adversarial_loss(discriminator\
									(fake_images.detach()), fake)
		d_loss = (real_loss + fake_loss) / 2

		# Backward pass and optimize
		d_loss.backward()
		optimizer_D.step()

		# -----------------
		# Train Generator
		# -----------------

		optimizer_G.zero_grad()

		# Generate a batch of images
		gen_images = generator(z)

		# Adversarial loss
		g_loss = adversarial_loss(discriminator(gen_images), valid)

		# Backward pass and optimize
		g_loss.backward()
		optimizer_G.step()

		# ---------------------
		# Progress Monitoring
		# ---------------------

		if (i + 1) % 100 == 0:
			print(
				f"Epoch [{epoch+1}/{num_epochs}]\
						Batch {i+1}/{len(dataloader)} "
				f"Discriminator Loss: {d_loss.item():.4f} "
				f"Generator Loss: {g_loss.item():.4f}"
			)

	# Save generated images for every epoch
	if (epoch + 1) % 10 == 0:
		with torch.no_grad():
			z = torch.randn(16, latent_dim, device=device)
			generated = generator(z).detach().cpu()
			grid = torchvision.utils.make_grid(generated,\
										nrow=4, normalize=True)
			plt.imshow(np.transpose(grid, (1, 2, 0)))
			plt.axis("off")
			plt.show()


Epoch [1/10]						Batch 100/1563 Discriminator Loss: 0.3521 Generator Loss: 1.1510
Epoch [1/10]						Batch 200/1563 Discriminator Loss: 0.6507 Generator Loss: 1.6493
Epoch [1/10]						Batch 300/1563 Discriminator Loss: 0.3588 Generator Loss: 1.5329
Epoch [1/10]						Batch 400/1563 Discriminator Loss: 1.0045 Generator Loss: 0.4291
Epoch [1/10]						Batch 500/1563 Discriminator Loss: 0.4074 Generator Loss: 1.2222
Epoch [1/10]						Batch 600/1563 Discriminator Loss: 0.6612 Generator Loss: 1.1582
Epoch [1/10]						Batch 700/1563 Discriminator Loss: 0.5995 Generator Loss: 0.8287
Epoch [1/10]						Batch 800/1563 Discriminator Loss: 0.6310 Generator Loss: 1.4543
Epoch [1/10]						Batch 900/1563 Discriminator Loss: 0.4429 Generator Loss: 1.7726
Epoch [1/10]						Batch 1000/1563 Discriminator Loss: 0.5140 Generator Loss: 2.2239
Epoch [1/10]						Batch 1100/1563 Discriminator Loss: 0.5708 Generator Loss: 0.8809
Epoch [1/10]						Batch 1200/1563 Discriminator Loss: 0.5351 Generator Loss: 1.4202
E